<a href="https://colab.research.google.com/github/adrianzevenster/ColabNotebooks/blob/main/Trend_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install -e git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper

In [ ]:
from google.cloud import bigquery
project_id = 'avantiate-task-21'
bq_ds = 'bigquery-public-data.google_trends.top_terms'

client = bigquery.Client(project=project_id)
ds_ref = client.dataset(bq_ds)

def gcp2df(sql):
  query = client.query(sql)
  results = query.result()
  return results.to_dataframe()


query = """ SELECT EXTRACT(YEAR from week) as search_year,
  EXTRACT(MONTH from week) as search_month,
  dma_name,
  dma_id, term,
  week,
  score
  FROM `bigquery-public-data.google_trends.top_terms`
  WHERE week >= '2020-01-01'
  LIMIT 23000"""

df = gcp2df(query)
print(df)

In [ ]:
#df = df.dropna(axis=0)
#print(df.isna())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df["refresh_date"] = pd.to_datetime(df["refresh_date"])
df["refresh_date"] = df["refresh_date"].dt.year
df["week"] = pd.to_datetime(df["refresh_date"])
df["week"] = df["week"].dt.month
df.head()



In [ ]:
df.corr()
df.dropna()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder_text_term = OrdinalEncoder()
df_term_col = df[["term"]]
df_term_encoded = ordinal_encoder_text_term.fit_transform(df_term_col)
ordinal_encoder_text_term.categories_

[array(['F1', 'PSG'], dtype=object)]

In [ ]:
ordinal_encoder_text_term.fit_transform(df[["dma_id"]])
ordinal_encoder_text_term.categories_
print(df['dma_id'])

In [ ]:
import numpy as np
def prep_data(target, win_X, window_y):
  X, y = [], []
  n = len(target)

  start_X = 0
  end_X = start_X + win_X
  start_y = end_X
  end_y = start_y + window_y
  for _ in range(n):
    if end_y < n:
      X.append(target[start_X:end_X])
      y.append(target[start_y:end_y])
    start_X += 1
    end_X = start_X + win_X
    start_y += 1
    end_y = start_y + window_y

  X = np.array(X)
  y = np.array(y)
  return X, y

prep_data(df, 12, 6)

In [ ]:
X = prep_data(df, 12, 6)[0]
y = prep_data(df, 12, 6)[1]
train = 100
X_train = X[:train]
y_train = y[:train]
x_valid = X[train:]
y_valid = y[train:]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


def model_fit(type, X_train, y_train, X_test, y_test, batch_size, epochs):
  model = Sequential()
  model.add(InputLayer(input_shape= (X_train.shape[1], )))

  if type == 'mlp':
    model.add(Reshape(target_shape=(X_train.shape[1], )))
    model.add(Dense(unit= 64, activation= 'relu'))

  if type == 'cnn':
    model.add(Reshape(target_shape=(X_train.shape[1], 1)))
    model.add(Conv1D(filters= 64, kernel_size=4, activation= 'relu'))
    model.add(MaxPool1D())
    model.add(Flatten())

  if type == 'lstm':
    model.add(Reshape(target_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=64, return_sequences= False))

  model.add(Dense(units= 64, activation= 'relu'))
  model.add(Dense(units= y_train.shape[0], activation= 'sigmoid'))

  model.compile(optimizer= 'adam', loss= 'mse')

  early_stopping = EarlyStopping(monitor= 'val_loss', patience= 10)
  model_checkpoint = ModelCheckpoint(filepath= 'content/sample_data/model.h5', save_best_only= True)
  callbacks = [early_stopping, model_checkpoint]


  model.fit(X, y, validation_data= (X_test, y_test),
            batch_size= batch_size, epochs= epochs, callbacks= callbacks)

  model.load_weights('model.h5')


  return model

model_fit(type, X_train, y_train, x_valid, y_valid, 42, 10)

Mounted at /content/drive


NameError: ignored